In [1]:
import scipy.stats as stats
from scipy.interpolate import BSpline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import shap


from efficient_kan import KAN
from models.mlp import MLP
from sklearn import svm


import torch
import torch.nn as nn

from kan.utils import create_dataset
from dataloaders.dataloader import FCMatrixDataset
from torch.utils.data import Dataset, DataLoader, Subset

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE = torch.device("cpu")


IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
import scipy.stats as stats
from scipy.interpolate import BSpline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import shap


from models.mlp import MLP
from models.GCN import GCN


from sklearn import svm




import torch
import torch.nn as nn

from kan.utils import create_dataset
from dataloaders.dataloader import FCMatrixDataset
from torch.utils.data import Dataset, DataLoader, Subset

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE = torch.device("cpu")

ModuleNotFoundError: No module named 'kan'

In [6]:
import wandb
wandb.login()

mlp_project = "severe_rds_v2_MLP"
# gcn_project = "severe_rds_v2_GCN"
kan_project = "severe_rds_v2_KAN_api"
svm_project = "severe_rds_v2_SVM"

# mlp_project = "rds_under_8_severe_v2_MLP"
# gcn_project = "rds_under_8_severe_v2_GCN"
# kan_project = "rds_under_8_severe_v2_KAN_api"
# svm_project = "rds_under_8_severe_v2_SVM"

api = wandb.Api()

# projects = {"MLP": api.project(mlp_project), "GCN": api.project(gcn_project), "SVM": api.project(svm_project)}
projects = {"MLP": api.project(mlp_project), "KAN": api.project(kan_project), "SVM": api.project(svm_project)}

print(projects.values())

all_sweeps = {}
for name, project in projects.items():
    all_sweeps[name] = [api.sweep(f"{project.name}/{sweep.id}") for sweep in project.sweeps()]

all_runs = {project_name: {sweep.id:sweep.runs for sweep in sweeps} for (project_name, sweeps) in all_sweeps.items()}


all_best_runs = {}

for name, sweeps in all_runs.items():
    project_sweeps = {}
    for (sweep_name, sweep) in sweeps.items():
        best_run = None
        best_acc = -1
        for run in sweep:

            ACC = run.summary.get("ACC")
            if ACC is not None and ACC > best_acc:
                best_run = run
                best_acc = ACC
        project_sweeps[sweep_name] = best_run
    all_best_runs[name] = project_sweeps


# for name, runs in all_runs.items():
#     print(runs)
#     best_run = None
#     best_acc = -1
#     for run in runs:
#         ACC = run.summary.get("ACC")
#         if ACC is not None and ACC > best_acc:
#             best_run = run
#             best_acc = ACC




dict_values([<Project laurens-devries/severe_rds_v2_MLP>, <Project laurens-devries/severe_rds_v2_KAN_api>, <Project laurens-devries/severe_rds_v2_SVM>])


In [7]:
for model in all_best_runs.keys():
    for run in all_best_runs[model].values():
        # print(model, run.summary.get("ACC"), run.summary.get("ACC std"), run.summary.get("AUC"), run.summary.get("AUC std"))
        # print the line above but multiply by 100 and round to get 1 decimal
        print(model, round(run.summary.get("ACC")*100, 1), round(run.summary.get("ACC std")*100, 1), round(run.summary.get("AUC")*100, 1), round(run.summary.get("AUC std")*100, 1))
        # print(f"{model}: {run.summary.get('ACC')}")
        print(run.config)
        print(model)
        print(run.name)

        test_acc = run.history()['test_acc']
        test_acc = [x for x in test_acc if str(x) != 'nan']
        # print(f"{model}: {test_acc}")

MLP 59.3 8.4 59.2 8.6
{'udi': 25751, 'mrmr': False, 'epochs': 25, 'dropout': 0.5635715546225145, 'n_layers': 1, 'optimizer': 'Adam', 'batch_size': 96, 'layer_size': 128, 'learning_rate': 0.00027037757777460727, 'hidden_dim_ratio': 0.125}
MLP
quiet-sweep-63
MLP 81.3 5.5 81.3 5.4
{'udi': 25751, 'mrmr': True, 'epochs': 25, 'dropout': 0.048468507432266786, 'n_layers': 3, 'optimizer': 'Adam', 'batch_size': 136, 'layer_size': 224, 'learning_rate': 0.0005185898072119435, 'hidden_dim_ratio': 0.25}
MLP
whole-sweep-32
MLP 83.9 6.6 83.8 6.7
{'udi': 25751, 'mrmr': True, 'epochs': 25, 'dropout': 0.5116486587155684, 'n_layers': 1, 'optimizer': 'Adam', 'batch_size': 104, 'layer_size': 160, 'learning_rate': 0.0006777949873387725, 'hidden_dim_ratio': 0.25}
MLP
earnest-sweep-111
KAN 56.5 12.2 56.4 12.1
{'udi': 25751, 'lamb': 0.1, 'mrmr': False, 'epochs': 25, 'n_layers': 1, 'grid_size': 3, 'optimizer': 'LBFGS', 'layer_size': 64, 'lamb_entropy': 2, 'learning_rate': 0.005955667351575727, 'hidden_dim_ratio'

In [5]:
r = all_best_runs['GCN'].values()
r = list(r)[0]
print(r.name)
print(r.config)
print(r.summary.get("ACC"))
r_accs = r.history()['test_acc']
r_accs = [x for x in r_accs if str(x) != 'nan']
r_accs

stellar-sweep-113
{'mrmr': False, 'epochs': 25, 'dropout': 0.15198839996723343, 'k_order': 3, 'k_degree': 2, 'optimizer': 'adam', 'batch_size': 152, 'learning_rate': 0.00014693506423494527}
0.5802536231884059


[0.625,
 0.6666666666666666,
 0.5416666666666666,
 0.5833333333333334,
 0.5416666666666666,
 0.5833333333333334,
 0.6521739130434783,
 0.4782608695652174,
 0.6521739130434783,
 0.4782608695652174]

In [7]:
best_mlp_s1=all_best_runs['GCN'].values()
best_mlp_s1 = list(best_mlp_s1)[1]

print(best_mlp_s1.summary.get("ACC"))
print(best_mlp_s1.summary.get("ACC std"))

mlp1_accs = best_mlp_s1.history()['test_acc']

mlp1_accs =[x for x in mlp1_accs if str(x) != 'nan']

print(mlp1_accs)


0.7331521739130434
0.05671215834009215
[0.75, 0.7916666666666666, 0.6666666666666666, 0.75, 0.6666666666666666, 0.75, 0.6521739130434783, 0.782608695652174, 0.6956521739130435, 0.8260869565217391]


In [8]:

t, p = stats.ttest_rel(mlp1_accs, r_accs)

print(t, p)
p2 = 2 * (1 - stats.t.cdf(abs(t), len(mlp1_accs)-1))
print(p2)

4.5920834003238475 0.0013051500194538922
0.0013051500194538601


In [59]:
svm_sweeps = all_runs["SVM"]

svm1 = svm_sweeps.values()
svm_runs = list(svm1)[1]
svm_runs2 = list(svm1)[1]

# get run named ethereal-sweep-387 from svm_runs

run = None
for r in svm_runs:
    if r.name == "frosty-sweep-23":
        run = r
        break

for r in svm_runs2:
    if r.name == "different-sweep-351":
        run2 = r
        break

svm_accs = run.history()['test_acc']
svm_accs2 = run2.history()['test_acc']

svm_accs =[x for x in svm_accs if str(x) != 'nan']
svm_accs2 =[x for x in svm_accs2 if str(x) != 'nan']
print(svm_accs)
print(svm_accs2)


# do t-test between svm_accs and svm_accs2
t, p = stats.ttest_rel(svm_accs, svm_accs2)

print(t, p)

[0.6213592233009708, 0.6019417475728155, 0.6407766990291263, 0.6796116504854369, 0.6116504854368932, 0.6310679611650486, 0.6213592233009708, 0.6407766990291263, 0.5631067961165048, 0.6407766990291263]
[0.6116504854368932, 0.6893203883495146, 0.6796116504854369, 0.6504854368932039, 0.6893203883495146, 0.6116504854368932, 0.6699029126213593, 0.5825242718446602, 0.5825242718446602, 0.6601941747572816]
-1.1739130434782599 0.27055743737938553


In [47]:
def visualize_kan(weight, scale_base):
    # define B-spline parameters
    grid_size = 3
    spline_order = 3
    weights = weight
    knot_vector = np.concatenate(([-1] * spline_order, np.linspace(-1, 1, grid_size), [1] * spline_order))

    t = np.linspace(-1, 1, 100)

    spline = BSpline(knot_vector, weights, spline_order)
    spline_values = spline(t)
    
    silu = nn.SiLU()
    bias = silu(torch.tensor(t))

    spline_values = spline_values + bias.numpy()*scale_base.detach().numpy()

    plt.figure(figsize=(4, 3))
    plt.plot(t, spline_values, label='B-spline curve')
    plt.scatter(np.linspace(-1, 1, len(weights)), weights, color='red', label='Control points')
    plt.title('B-spline Curve')
    plt.xlabel('t')
    plt.ylabel('Value')
    plt.legend()
    plt.grid(True)
    plt.show()

def calculate_layer_overlap(model, feature, layer):
    weights = model.layers[layer].spline_weight # out, in, grid_size

    print(dir(model.layers[layer]))




lunar-sweep-101


In [46]:
best_kans = all_best_runs['KAN'].values()
best_kans = list(best_kans)

mrmr_adam_kan = best_kans[3]
gs = mrmr_adam_kan.config['grid_size']
model = KAN([55,8,2], gs)
# cp = torch.load(f"models/saved/KAN/{mrmr_adam_kan.name}.pth", map_location=DEVICE)
# model.load_state_dict(cp)

# get the first layer weights

calculate_layer_overlap(model, None, 0)




['B_batch', 'T_destination', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_backward_pre_hooks', '_buffers', '_call_impl', '_compiled_call_impl', '_forward_hooks', '_forward_hooks_always_called', '_forward_hooks_with_kwargs', '_forward_pre_hooks', '_forward_pre_hooks_with_kwargs', '_get_backward_hooks', '_get_backward_pre_hooks', '_get_name', '_is_full_backward_hook', '_load_from_state_dict', '_load_state_dict_post_hooks', '_load_state_dict_pre_hooks', '_maybe_warn_non_full_backward_hook', '_modules', '_named_members', '_non_persistent_buffers_set', '_parameters', '_register_load_state_d

In [8]:
ds = "data/csv/severe_rds_v2.csv"
u8_ds = "data/csv/rds_under_8_severe_v2.csv"
data_dir = "data/fetched/25751/raw"

mrmr_features = np.array([1140, 689, 427, 122, 139, 765, 907, 1384, 22, 1293, 1449, 492, 1440, 499, 1316, 1318, 135, 879, 886, 223, 1455, 676, 1136, 1464, 70, 1462, 1386, 939, 111, 413, 10, 1403, 1027, 547, 395, 1210, 942, 501, 45, 425, 638, 505, 26, 1409, 1448, 605, 232, 1459, 821, 1394, 1376, 809, 1163, 216, 791]
            )

mrmr_features_u8 = np.array([1035, 382, 967, 1316, 34, 852, 761, 480, 875, 1357, 1203, 686, 415, 248, 1230, 1370, 1275, 466, 1274, 126, 840, 1470, 448, 629, 1292, 922, 617, 168, 946, 1372, 131, 219, 1247, 413, 72, 496, 880, 275, 863, 11, 571, 147, 877, 843, 92, 124, 1157, 1457, 249, 1126, 741, 1289, 1270, 1453, 742]
            )


mrmr_dataset = FCMatrixDataset(ds, data_dir, "25751", None, mrmr=mrmr_features)
mrmr_u8_dataset = FCMatrixDataset(u8_ds, data_dir, "25751", None, mrmr=mrmr_features_u8)
dataset = FCMatrixDataset(ds, data_dir, "25751", None)

models = {"MLP": MLP, "KAN":KAN, "SVM": "SVM"}

for model_type in all_best_runs.keys():
    for run in all_best_runs[model_type].values():
        c = run.config
        # print(c)
        # print(model_type)

        if model_type == "GCN":
            # jump to next iteration
            continue
        if model_type == "MLP" or model_type == "KAN":
            input_features = 55 if c['mrmr'] else 1485

            cp = torch.load(f"models/saved/{model_type}/{run.name}.pth", map_location=DEVICE)

        if model_type == "MLP":
            hidden_dims = [int(c['layer_size'] * c['hidden_dim_ratio'] ** i) for i in range(c['n_layers'])]
            model = MLP(input_features, hidden_dims, 2, c['dropout']).to(DEVICE)
            model.load_state_dict(cp)

        if model_type == "KAN":
            hidden_dims = [int(c['layer_size'] * c['hidden_dim_ratio'] ** i) for i in range(c['n_layers'])]
            dims = [input_features] + hidden_dims + [2]
            model = KAN(dims, c['grid_size']).to(DEVICE)
            model.load_state_dict(cp)
            print(f"models/saved/{model_type}/{run.name}.pth")
            print(c["mrmr"], c["optimizer"])

        if model_type == "SVM":
            model = svm.SVC(kernel= c['kernel'], C=c['C'], gamma=c['gamma'], degree=c['degree'])

        if c["mrmr"] == True:
            X = torch.tensor(np.array([np.array(i[0]) for i in mrmr_dataset]))
            y = torch.tensor(np.array([i[1] for i in mrmr_dataset]))
        else:
            X = torch.tensor(np.array([np.array(i[0]) for i in dataset]))
            y = torch.tensor(np.array([i[1] for i in dataset]))


        # model.fit(X, y)
        # explainer = shap.DeepExplainer(model.predict, X)
        # shap_values = explainer.shap_values(np.array(X)[:5])
        # # shap.summary_plot(shap_values)

        # shap_v_summed = np.mean(np.abs(shap_values[0]), axis=0) + np.mean(np.abs(shap_values[1]), axis=0)
        # sorted_idx = np.argsort(shap_v_summed)[::-1]
        # print(sorted_idx)



        # print(run.name)
        



models/saved/KAN/honest-sweep-45.pth
False LBFGS
models/saved/KAN/glorious-sweep-83.pth
False Adam
models/saved/KAN/daily-sweep-67.pth
True LBFGS
models/saved/KAN/lunar-sweep-101.pth
True Adam


'elated-sweep-160'